### NOCHES

In [1]:
import pandas as pd

# Definir las horas de inscripción (8:00 am - 5:00 pm)
hora_inicio_inscripcion = 8 * 60  # 8:00 am en minutos (480 minutos)
hora_fin_inscripcion = 17 * 60  # 5:00 pm en minutos (1020 minutos)


# Función corregida para calcular la duración en horas activas (8:00 am a 5:00 pm)
def calcular_duration_en_horas_activas(first_enroll_time, fill_time):
    # Definir los horarios de inicio y fin de inscripción
    start_hour = 8
    end_hour = 17

    # Si el tiempo de llenado es antes de la inscripción, no hay tiempo válido
    if fill_time < first_enroll_time:
        return 0
    
    #############Calcular la duración en el mismo día
    def calcular_horas_dia(dia_inicio, dia_fin):
        # Obtener el máximo entre la hora de inicio y el horario de inicio permitido
        inicio_efectivo = max(dia_inicio.hour + dia_inicio.minute / 60, start_hour)
        # Obtener el mínimo entre la hora de fin y el horario de fin permitido
        fin_efectivo = min(dia_fin.hour + dia_fin.minute / 60, end_hour)
        # Si el horario efectivo es válido (en el rango de 8 am a 5 pm)
        return max(0, fin_efectivo - inicio_efectivo)

    total_minutes = 0

    ############## Si ambos tiempos están en el mismo día ######varios diasss
    if first_enroll_time.date() == fill_time.date():
        total_minutes += calcular_horas_dia(first_enroll_time, fill_time) * 60
    else:
        # Calcular las horas para el primer día
        first_day_end = pd.Timestamp(first_enroll_time.year, first_enroll_time.month, first_enroll_time.day, end_hour, 0, 0)
        total_minutes += calcular_horas_dia(first_enroll_time, first_day_end) * 60

        # Calcular las horas para los días intermedios
        next_day = first_enroll_time + pd.Timedelta(days=1)
        while next_day.date() < fill_time.date():
            # Si el siguiente día no es fin de semana
            if next_day.weekday() < 5:  # weekday() devuelve 0 para lunes y 4 para viernes
                total_minutes += (end_hour - start_hour) * 60
            next_day += pd.Timedelta(days=1)

        # Calcular las horas para el último día
        fill_day_start = pd.Timestamp(fill_time.year, fill_time.month, fill_time.day, start_hour, 0, 0)
        total_minutes += calcular_horas_dia(fill_day_start, fill_time) * 60

    return total_minutes

In [2]:
import pandas as pd

# Caso 1: Inscripción y llenado el mismo día dentro del horario de inscripción
# Descripción: La inscripción comienza a las 9:00 am y el curso se llena a las 2:00 pm del mismo día.
# Expectativa: La duración es de 5 horas (300 minutos).
first_enroll_time = pd.Timestamp("2024-09-10 09:00:00")
fill_time = pd.Timestamp("2024-09-10 14:00:00")
print(calcular_duration_en_horas_activas(first_enroll_time, fill_time))  # Esperado: 300.0 minutos


# Caso 2: Inscripción comienza antes del horario de inscripción y se llena dentro del horario
# Descripción: La inscripción comienza a las 7:30 am y el curso se llena a las 12:00 pm del mismo día.
# Expectativa: Solo se cuenta desde las 8:00 am, por lo que la duración es de 4 horas (240 minutos).
first_enroll_time = pd.Timestamp("2024-09-10 07:30:00")
fill_time = pd.Timestamp("2024-09-10 12:00:00")
print(calcular_duration_en_horas_activas(first_enroll_time, fill_time))  # Esperado: 240.0 minutos


# Caso 3: Inscripción comienza durante el horario y se llena fuera del horario (después de las 5:00 pm)
# Descripción: La inscripción comienza a las 10:00 am y el curso se llena a las 6:30 pm.
# Expectativa: Solo se cuenta hasta las 5:00 pm, por lo que la duración es de 7 horas (420 minutos).
first_enroll_time = pd.Timestamp("2024-09-10 10:00:00")
fill_time = pd.Timestamp("2024-09-10 18:30:00")
print(calcular_duration_en_horas_activas(first_enroll_time, fill_time))  # Esperado: 420.0 minutos


# Caso 4: Inscripción comienza y se llena completamente fuera del horario (antes de las 8:00 am y después de las 5:00 pm)
# Descripción: La inscripción comienza a las 7:00 am y el curso se llena a las 7:30 pm.
# Expectativa: No se cuenta ningún tiempo porque ambas horas están fuera del horario de inscripción (0 minutos).
first_enroll_time = pd.Timestamp("2024-09-10 07:00:00")
fill_time = pd.Timestamp("2024-09-10 19:30:00")
print(calcular_duration_en_horas_activas(first_enroll_time, fill_time))  # Esperado: 540 minutos


# Caso 5: Inscripción comienza un día y se llena al día siguiente
# Descripción: La inscripción comienza a las 4:00 pm el primer día y se llena a las 9:00 am del día siguiente.
# Expectativa: La duración total es de 1 hora el primer día (4:00 pm a 5:00 pm) y 1 hora el segundo día (8:00 am a 9:00 am), por lo que la duración es de 2 horas (120 minutos).
first_enroll_time = pd.Timestamp("2024-09-10 16:00:00")
fill_time = pd.Timestamp("2024-09-11 09:00:00")
print(calcular_duration_en_horas_activas(first_enroll_time, fill_time))  # Esperado: 120.0 minutos


# Caso 6: Inscripción dura varios días, llenado dentro del horario en un día posterior
# Descripción: La inscripción comienza a las 11:00 am del primer día y el curso se llena a las 10:00 am dos días después.
# Expectativa: Se cuenta 6 horas el primer día (11:00 am a 5:00 pm), 9 horas el segundo día (8:00 am a 5:00 pm), y 2 horas el tercer día (8:00 am a 10:00 am), lo que da un total de 17 horas (1020 minutos).
first_enroll_time = pd.Timestamp("2024-09-10 11:00:00")
fill_time = pd.Timestamp("2024-09-12 10:00:00")
print(calcular_duration_en_horas_activas(first_enroll_time, fill_time))  # Esperado: 1020.0 minutos




300.0
240.0
420.0
540
120.0
1020.0


In [3]:
first_enroll_time = pd.Timestamp("2024-07-18 12:06:32")
fill_time = pd.Timestamp("2024-07-19 00:17:03")
print(calcular_duration_en_horas_activas(first_enroll_time, fill_time))  


294.0


In [4]:
import pandas as pd
import numpy as np

# Función corregida para calcular la duración en horas activas (8:00 am a 5:00 pm)
def calcular_duration_en_horas_activas(first_enroll_time, fill_time):
    # Definir los horarios de inicio y fin de inscripción
    start_hour = 8
    end_hour = 17

    # Si el tiempo de llenado es antes de la inscripción, no hay tiempo válido
    if fill_time < first_enroll_time:
        return 0
    
    # Calcular la duración en el mismo día
    def calcular_horas_dia(dia_inicio, dia_fin):
        # Obtener el máximo entre la hora de inicio y el horario de inicio permitido
        inicio_efectivo = max(dia_inicio.hour + dia_inicio.minute / 60, start_hour)
        # Obtener el mínimo entre la hora de fin y el horario de fin permitido
        fin_efectivo = min(dia_fin.hour + dia_fin.minute / 60, end_hour)
        # Si el horario efectivo es válido (en el rango de 8 am a 5 pm)
        return max(0, fin_efectivo - inicio_efectivo)

    total_minutes = 0

    # Si ambos tiempos están en el mismo día
    if first_enroll_time.date() == fill_time.date():
        total_minutes += calcular_horas_dia(first_enroll_time, fill_time) * 60
    else:
        # Calcular las horas para el primer día
        first_day_end = pd.Timestamp(first_enroll_time.year, first_enroll_time.month, first_enroll_time.day, end_hour, 0, 0)
        total_minutes += calcular_horas_dia(first_enroll_time, first_day_end) * 60

        # Calcular las horas para los días intermedios
        next_day = first_enroll_time + pd.Timedelta(days=1)
        while next_day.date() < fill_time.date():
            # Si el siguiente día no es fin de semana
            if next_day.weekday() < 5:  # weekday() devuelve 0 para lunes y 4 para viernes
                total_minutes += (end_hour - start_hour) * 60
            next_day += pd.Timedelta(days=1)

        # Calcular las horas para el último día
        fill_day_start = pd.Timestamp(fill_time.year, fill_time.month, fill_time.day, start_hour, 0, 0)
        total_minutes += calcular_horas_dia(fill_day_start, fill_time) * 60

    return total_minutes


# Caso de prueba proporcionado
first_enroll_time = pd.Timestamp("2024-07-18 12:06:32")
fill_time = pd.Timestamp("2024-07-19 00:17:03")
print(calcular_duration_en_horas_activas(first_enroll_time, fill_time))  # Esperado: 297.0 minutos


294.0
